In [5]:
# 1. tokenization in tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from keras.layers import Embedding, Dense, Bidirectional, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
# to deal with text input of different size we need to add a few things. This will enables training in neural networks
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam



In [6]:
tokenizer=Tokenizer()
data=f"""In the town of Athy one Jeremy Lanigan \n
Battered away til he hadnt a pound. \n
His father died and made him a man again \n
Left him a farm and ten acres of ground. \n

He gave a grand party for friends and relations \n
Who he didnt forget him when come to the wall, \n
And if youll but listen Ill make your eyes glisten \n
Of the rows and the ructions of Lanigan's Ball. \n

Myself to be sure got free invitation, \n
For all the nice girls and boys I might ask, \n
And just in a minute both friends and relations \n
Were dancing round merry as bees round a cask. \n

Judy ODaly, that nice little milliner, \n
She tipped me a wink for to give her a call, \n
And I soon arrived with Peggy McGilligan \n
Just in time for Lanigan's Ball. \n"""
corpus=data.lower().split("\n")
tokenizer.fit_on_texts(corpus)

total_words=len(tokenizer.word_index) +1 #we are adding +1 since we are using an out of word index
# When generating text we don't use a validation set we use all the data that we have
# Therefore when we tokenize the entire data set there will be no out of vocabulary token
# we will need padding hence the reason for that extra token
print(total_words)

96


In [7]:
# we now turn the tokenized sentences into a set of training data
# NB : There is a difference between how we prepare data for classification and data for text generation
import numpy as np
input_sequences=[] # empty array of input sequences that will be populated as we go along
for line in corpus:
  token_list=tokenizer.texts_to_sequences([line])[0] # text sequences one line at a time
  for i in range(1,len(token_list)) : # go through the token list and generate n-grams It will help  train the model know the next number in the sequence
    n_gram_sequence=token_list[:i+1]
    input_sequences.append(n_gram_sequence)

# we need to pad the sentences hence will get the longest sentence and use it to pad the rest
max_sequence_len=max([len(x) for x in input_sequences])
# next, we pad everything with a zero
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
# now we take everything except the last value as our X and the last value as our Y
xs=input_sequences[:,:-1]
labels=input_sequences[:,-1]
# we then one hot our Y and make it categorical
ys=tf.keras.utils.to_categorical(labels,num_classes=total_words)


In [9]:
# training the neural network on this data
model=Sequential()
model.add(Embedding(total_words,240,input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words,activation='softmax'))
adam=Adam(learning_rate=0.01)

In [10]:
# compile the model
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
history=model.fit(xs,ys,epochs=100,verbose=1)

Epoch 1/100
4/4 [==============================] - 7s 274ms/step - loss: 4.5791 - accuracy: 0.0417
Epoch 2/100
4/4 [==============================] - 1s 96ms/step - loss: 4.4116 - accuracy: 0.0750
Epoch 3/100
4/4 [==============================] - 0s 65ms/step - loss: 4.0101 - accuracy: 0.1500
Epoch 4/100
4/4 [==============================] - 1s 122ms/step - loss: 3.3328 - accuracy: 0.1917
Epoch 5/100
4/4 [==============================] - 0s 65ms/step - loss: 2.7018 - accuracy: 0.2250
Epoch 6/100
4/4 [==============================] - 0s 121ms/step - loss: 1.9649 - accuracy: 0.3667
Epoch 7/100
4/4 [==============================] - 0s 125ms/step - loss: 1.5509 - accuracy: 0.4917
Epoch 8/100
4/4 [==============================] - 0s 66ms/step - loss: 1.1074 - accuracy: 0.6167
Epoch 9/100
4/4 [==============================] - 0s 66ms/step - loss: 0.7797 - accuracy: 0.7333
Epoch 10/100
4/4 [==============================] - 0s 124ms/step - loss: 0.5699 - accuracy: 0.8833
Epoch 11/100
4